In [85]:
import pandas as pd
import numpy as np
import csv
from decimal import Decimal
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))



In [86]:
def get_ngram(text, n):

    # print(text)
    words = text

    return [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]

# def get_ngram(df):

#     for line in df['abstract']:
#         words = [word for word in line.split() if word not in stopWords]
#         bigram = ngram(words, 2)
        
def get_vocab(df):
    stopWords = set(stopwords.words('english'))
    vocab = {word for para in df['abstract'] for word in para.split() if word not in stopWords}
    # for line in df['abstract']:
    #     words = [word for word in line.split() if word not in stopWords]
    #     bigram = get_ngram(words, 2)
    #     vocab.update(bigram)
        # if 'regulatory control' in bigram:
        #     print(bigram)


    # vocabulary = set() 
    # vocabulary = {word for line in df['abstract'] for word in line.split() if word not in stopWords}
    # print(len(vocab))
    return vocab

def get_prior_probability(df):
    prior_probability = df['target'].value_counts(normalize=True).to_dict()
    for key, value in prior_probability.items():
        prior_probability[key] = Decimal(str(value))
    return prior_probability 

def get_conditional_probability(df,vocab):
    count = 0
    conditional_probability = dict()
    # ngram =[]
    for row in df.itertuples(index = 0): 
        count+=1
        # print(count)
        line = row.abstract
        target = row.target
        # print(target)
        words = [word for word in line.split() if word not in stopWords]
        
        # ngram_1 =get_ngram(words,2)

        # print(ngram_1)
        
        # for word in words + ngram_1:
        for word in words :
            if word == "regulatory control":
                print(word)
            
            if word in conditional_probability.keys():
                if target in conditional_probability[word].keys():
                    conditional_probability[word][target] += 1
                else:
                    conditional_probability[word][target] = 1
            else:
                conditional_probability[word] = {"B": Decimal(0), "E":Decimal(0), "A":Decimal(0), "V":Decimal(0)}
                conditional_probability[word][target] += 1
    return conditional_probability

def get_conditional_probability_df(conditional_probability):

    data = []
    for word, targets in conditional_probability.items():
        
        row = {'word': word}
        
        for i, (target, count) in enumerate(targets.items()):
            row [target] = count
        # append the row to the data list
        data.append(row)
    word_orrurance_table = pd.DataFrame(data)
    word_orrurance_table.fillna(0, inplace=True)
    return word_orrurance_table



In [87]:



def train(df):
    vocab = get_vocab(df)
    prior_probability = get_prior_probability(df)
    conditional_probability_dict = get_conditional_probability(df,vocab)
    conditional_probability_df = get_conditional_probability_df(conditional_probability_dict)
    return vocab, prior_probability, conditional_probability_dict,conditional_probability_df

def naive_Bayes(vocab, prior_probability, conditional_probability_dict,conditional_probability_df,testing):
    predictions = []
    row_nm = 0
    for row in testing.itertuples(index = 0):
        row_nm += 1
        # print(row_nm)
        line = row.abstract
        prediction = predict(vocab,prior_probability,conditional_probability_dict,conditional_probability_df,line)
        # print(prediction, row.target)
        predictions.append(prediction)
    # testing['prediction'] = predictions
    return predictions

def predict(vocab,prior_probability,conditional_probability_dict,conditional_probability_df,line):
    words = line.split()
    # ngrams = get_ngram(words, 2)
    # print(words)
    prob_B = prior_probability['B'].ln()
    prob_E = prior_probability['E'].ln()
    prob_A = prior_probability['A'].ln()
    prob_V = prior_probability['V'].ln()

    total_B = conditional_probability_df['B'].sum()
    total_E = conditional_probability_df['E'].sum()
    total_A = conditional_probability_df['A'].sum()
    total_V = conditional_probability_df['V'].sum()
    # for word in words + ngrams:
    for word in words:
        if word in vocab:
            prob_B += (conditional_probability_dict[word]['B'] + 1).ln() - (Decimal(total_B + len(vocab))).ln()
            prob_A += (conditional_probability_dict[word]['A'] + 1).ln() - (Decimal(total_A + len(vocab))).ln()
            prob_V += (conditional_probability_dict[word]['V'] + 1).ln() - (Decimal(total_V + len(vocab))).ln()
            prob_E += (conditional_probability_dict[word]['E'] + 1).ln() - (Decimal(total_E + len(vocab))).ln()
    # print(prob_B,prob_E,prob_A,prob_V)
    if prob_B > prob_E and prob_B > prob_A and prob_B > prob_V:
        return 'B'
    elif prob_E > prob_B and prob_E > prob_A and prob_E > prob_V:
        return 'E'
    elif prob_A > prob_B and prob_A > prob_E and prob_A > prob_V:
        return 'A'
    elif prob_V > prob_B and prob_V > prob_E and prob_V > prob_A:
        return 'V'
    else:
        return 'E'

# vocab, prior_probability, conditional_probability_dict,conditional_probability_df = train(df)


### Validation Set

In [88]:
from sklearn.model_selection import train_test_split

# def split_train_test(df):
#     df = df.sample(frac=1, random_state=32)

#     count_rows = len(df.index)
#     count_train_set = round(count_rows * 0.7)

#     train_set_df = df.iloc[:count_train_set]
#     validation_set_df = df.iloc[count_train_set:]

#     return train_set_df,validation_set_df



# df = pd.read_csv("trg.csv")
# df = df.rename({'class':'target'},axis=1)
# training, testing = train_test_split(df, test_size=0.3, random_state=42)
# # training, testing = split_train_test(df)
# vocab, prior_probability, conditional_probability_dict,conditional_probability_df = train(training)
# prediction_row = naive_Bayes(vocab, prior_probability, conditional_probability_dict,conditional_probability_df, testing)
# print(prediction_row)
# # print(testing['target'])
# validation_acc = (testing['target'] == prediction_row).mean()
# print(f"Training acc: {validation_acc:.3f}")

### Testing Set

In [89]:
df = pd.read_csv("trg.csv")
df = df.rename({'class':'target'},axis=1)
testing = pd.read_csv("tst.csv")
test_df = pd.read_csv("tst.csv")
vocab, prior_probability, conditional_probability_dict,conditional_probability_df = train(df)

prediction_row = naive_Bayes(vocab, prior_probability, conditional_probability_dict,conditional_probability_df, test_df)

with open("tst.csv","rt") as source:
    rdr = csv.reader(source)
    c = -1
    with open("raar518.csv","wt", newline='') as result:
        wtr = csv.writer( result )
        for r in rdr:
            if (c==-1): wtr.writerow( (r[0], "class") ) #add csv header
            else: wtr.writerow( (r[0], prediction_row[c]) )
            c+=1